## Create new nodal cost and nodal energy balance files

Due to a bug in the PyPSA-Eur cope, the nodal CSV files do not have assigned nodes for geothermal heat generation and direct geothermal heat utilisation. So we create new nodak cost and nodal energy balance files with this code, based on the network NC files.

In [ ]:
import pypsa
import pandas as pd
from pathlib import Path

output_folder = Path(r"C:\Users\Johannes\Documents\final-results\new_cost_csv")
output_folder.mkdir(parents=True, exist_ok=True)

# Fix missing bus locations
def patch_assign_locations(n):
    # 1: Patch bus locations
    missing = n.buses["location"].isna() | (n.buses["location"] == "")
    if missing.any():
        print(f" Patching {missing.sum()} buses with missing or empty location.")
        n.buses.loc[missing, "location"] = (
            n.buses.loc[missing].index.to_series()
            .str.extract(r"^([A-Z]{2}\d+)")[0]
        )

    # 2: Patch one-port components (generators, stores, loads, etc.)
    for c in n.iterate_components(n.one_port_components):
        df = c.df
        if "location" not in df.columns:
            df["location"] = ""  # Create column if missing
        is_missing = df["location"].isna() | (df["location"] == "")
        if is_missing.any():
            print(f" Patching {is_missing.sum()} {c.name} entries with missing/empty location.")
            df.loc[is_missing, "location"] = df.loc[is_missing, "bus"].map(n.buses["location"])

    # 3: Patch branch components (links and lines)
    for c in n.iterate_components(n.branch_components):
        df = c.df
        if "location" not in df.columns:
            df["location"] = ""  # Create column if missing
        bus_cols = df.filter(regex="^bus")
        locs = bus_cols.apply(lambda col: col.map(n.buses["location"]))
        df["location"] = locs.bfill(axis=1).iloc[:, 0]
        still_missing = df["location"].isna() | (df["location"] == "")
        if still_missing.any():
            print(f" {still_missing.sum()} {c.name} entries still have missing/empty location.")

# Calculate nodal costs
def calculate_nodal_costs(n):
    grouper = ["location", "carrier"]
    costs = pd.concat({
        "capital": n.statistics.capex(groupby=grouper),
        "marginal": n.statistics.opex(groupby=grouper),
    })
    costs.index.names = ["cost", "component", "location", "carrier"]
    return costs

# Calculate nodal energy balance
def calculate_nodal_energy_balance(n: pypsa.Network) -> pd.Series:
    """
    Calculate the regional energy balances (positive values for supply, negative values for consumption) for each technology carrier and bus carrier based on the location bus attribute.

    Returns
    -------
    pd.Series
        MultiIndex Series with levels ["component", "carrier", "location", "bus_carrier"]

    Examples
    --------
    >>> eb = calculate_nodal_energy_balance(n)
    >>> eb.xs(("AC", "BE0 0"), level=["bus_carrier", "location"])
    """
    return n.statistics.energy_balance(groupby=["carrier", "location", "bus_carrier"])

# Save as new CSV files
network_files = list(Path(r"C:\Users\Johannes\Documents\final-results\networks").glob("*.nc"))
for net_file in network_files:
    n = pypsa.Network(net_file)
    patch_assign_locations(n)
    for comp_name in ["generators", "links", "stores", "loads"]:
        df = getattr(n, comp_name)
        if "location" in df.columns:
            missing = df[df["location"].isna() | (df["location"] == "")]
            if not missing.empty:
                print(f"\n {comp_name.upper()} with missing/empty location:")
                print(missing[["carrier", "bus", "location"]].head())

    n.statistics.set_parameters(nice_names=False, drop_zero=False)

    nodal_costs = calculate_nodal_costs(n)
    energy_balance = calculate_nodal_energy_balance(n)
    stem = net_file.stem  # for example "base_s_16__3H_2050-htates"
    # Remove the "base_" prefix to match the original naming convention
    if stem.startswith("base_"):
        stem = stem.replace("base_", "")
    # Save files in a new folder
    output_file = Path(r"C:\Users\Johannes\Documents\final-results\new_cost_csv") / f"nodal_costs_{stem}.csv"
    nodal_costs.to_csv(output_file)

    output_file2 = Path(r"C:\Users\Johannes\Documents\final-results\new_cost_csv") / f"{net_file.stem}_energy_balance.csv"
    energy_balance.to_csv(output_file2)


INFO:pypsa.io:Imported network base_s_12__3H_2050.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


 Patching 12 buses with missing or empty location.
 Patching 139 Load entries with missing/empty location.
 Patching 5 StorageUnit entries with missing/empty location.
 Patching 158 Generator entries with missing/empty location.
 Patching 100 Store entries with missing/empty location.


INFO:pypsa.io:Imported network base_s_16__3H_2030-htates-2.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


 Patching 16 buses with missing or empty location.
 Patching 186 Load entries with missing/empty location.
 Patching 5 StorageUnit entries with missing/empty location.
 Patching 218 Generator entries with missing/empty location.
 Patching 130 Store entries with missing/empty location.


INFO:pypsa.io:Imported network base_s_16__3H_2030-htates-3.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


...

 Patching 20 buses with missing or empty location.
 Patching 227 Load entries with missing/empty location.
 Patching 6 StorageUnit entries with missing/empty location.
 Patching 242 Generator entries with missing/empty location.
 Patching 160 Store entries with missing/empty location.


INFO:pypsa.io:Imported network base_s_60__3H_2050-geo2.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


 Patching 58 buses with missing or empty location.
 Patching 665 Load entries with missing/empty location.
 Patching 40 StorageUnit entries with missing/empty location.
 Patching 684 Generator entries with missing/empty location.
 Patching 479 Store entries with missing/empty location.


INFO:pypsa.io:Imported network base_s_60__3H_2050-geo3.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


 Patching 58 buses with missing or empty location.
 Patching 665 Load entries with missing/empty location.
 Patching 40 StorageUnit entries with missing/empty location.
 Patching 684 Generator entries with missing/empty location.
 Patching 479 Store entries with missing/empty location.


## Create heat balance plots based on the plot_balance_timeseries.py script

New plots are made to ensure HT-ATES is always plotted with colours that stand out. This was also necessary to include geothermal heat in the heat balance plots.

In [ ]:
import os
import yaml
import pypsa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tqdm import tqdm
from functools import partial
from multiprocessing import Pool

# Load pypsa network
os.chdir(r"C:\Users\Johannes\Documents\final-results\networks")
n = pypsa.Network("base_s_16__3H_2050-htates.nc")

# Load configuration file
with open(r"C:\Users\Johannes\PypsaProject\pypsa-eur-htates\results\batch_1207\2050-htates\configs\config.base_s_16__3H_2050.yaml", encoding="windows-1252") as f:
    full_config = yaml.safe_load(f)

config = full_config["plotting"]["balance_timeseries"]

# Add missing geothermal and HT-ATES-related carriers to the heat group
additional_heat_carriers = [
    "urban central ht_ates charger",
    "urban central ht_ates discharger",
    "urban central geothermal heat pump",
    "urban central geothermal heat direct utilisation"
]

# Avoid duplicates
existing_heat_group = config["carrier_groups"].get("heat", [])
for carrier in additional_heat_carriers:
    if carrier not in existing_heat_group:
        existing_heat_group.append(carrier)

config["carrier_groups"]["heat"] = existing_heat_group

# Also add to full list of carriers
for carrier in additional_heat_carriers:
    if carrier not in config["carriers"]:
        config["carriers"].append(carrier)

# Filter for only NL nodes
nl_buses = n.buses[n.buses.index.str.startswith("NL")].index
n.remove("Load", n.loads[~n.loads.bus.isin(nl_buses)].index)
n.remove("Store", n.stores[~n.stores.bus.isin(nl_buses)].index)
n.remove("Generator", n.generators[~n.generators.bus.isin(nl_buses)].index)
n.remove("Link", n.links[~n.links.bus0.isin(nl_buses) & ~n.links.bus1.isin(nl_buses)].index)
n.remove("Bus", n.buses[~n.buses.index.isin(nl_buses)].index)
n.remove("StorageUnit", n.storage_units[~n.storage_units.bus.isin(nl_buses)].index)

# Plotting setup 
plt.style.use("bmh")
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)

# Plotting functions
def plot_stacked_area_steplike(ax, df, colors={}):
    if isinstance(colors, pd.Series):
        colors = colors.to_dict()
    df_cum = df.cumsum(axis=1)
    previous = np.zeros_like(df_cum.iloc[:, 0].values)
    for col in df_cum.columns:
        ax.fill_between(
            df_cum.index,
            previous,
            df_cum[col],
            step="pre",
            linewidth=0,
            color=colors.get(col, "grey"),
            label=col,
        )
        previous = df_cum[col].values

def setup_time_axis(ax, timespan):
    if timespan > pd.Timedelta(weeks=5):
        ax.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%e\n%b"))
        ax.xaxis.set_minor_locator(mdates.MonthLocator(bymonthday=15))
        ax.xaxis.set_minor_formatter(mdates.DateFormatter("%e"))
    else:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY))
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%e\n%b"))
        ax.xaxis.set_minor_locator(mdates.DayLocator())
        ax.xaxis.set_minor_formatter(mdates.DateFormatter("%e"))
    ax.tick_params(axis="x", which="minor", labelcolor="grey")

def plot_energy_balance_timeseries(
    df,
    time=None,
    ylim=None,
    resample=None,
    rename={},
    preferred_order=[],
    ylabel="",
    colors={},
    max_threshold=0.0,
    mean_threshold=0.0,
    directory="",
):
    if time is not None:
        df = df.loc[time]
    exempt_from_other = {
        "urban central ht_ates charger",
        "urban central ht_ates discharger",
        "urban central geothermal heat pump",
        "rural heat"
    }

    techs_below_threshold = [
        tech for tech in df.columns
        if tech not in exempt_from_other and
        (df[tech].abs().max() < max_threshold) and
        (df[tech].abs().mean() < mean_threshold)
    ]
    if techs_below_threshold:
        rename.update({tech: "other" for tech in techs_below_threshold})
        colors["other"] = "grey"

    rename.update({
        "urban central ht_ates charger": "urban central HT-ATES charger",
        "urban central ht_ates discharger": "urban central HT-ATES discharger",
        "urban central geothermal heat pump": "urban central geothermal heat pump"
    })

    if rename:
        df = df.T.groupby(df.columns.map(lambda a: rename.get(a, a))).sum().T

    if resample is not None:
        df = df.resample("1h").ffill().resample(resample).mean()

    order = (df / df.max()).var().sort_values().index
    if preferred_order:
        order = pd.Index(preferred_order).intersection(order).append(order.difference(preferred_order))
    df = df.loc[:, order]

    pos = df.where(df > 0).fillna(0.0)
    neg = df.where(df < 0).fillna(0.0)

    fig, ax = plt.subplots(figsize=(10, 4.5), layout="constrained")
    fig.patch.set_facecolor("white")
    ax.set_facecolor("white")
    plot_stacked_area_steplike(ax, pos, colors)
    plot_stacked_area_steplike(ax, neg, colors)

    plt.xlim((df.index[0], df.index[-1]))
    setup_time_axis(ax, df.index[-1] - df.index[0])

    ax.grid(axis="y")
    ax.axhline(0, color="grey", linewidth=0.5)

    if ylim is None:
        ylim = np.ceil(max(-neg.sum(axis=1).min(), pos.sum(axis=1).max()) / 50) * 50
    plt.ylim([-ylim, ylim])

    plt.ylabel(f"{ylabel} balance [GW]")
    handles, labels = ax.get_legend_handles_labels()
    half = int(len(handles) / 2)
    legend = fig.legend(handles[:half], labels[:half], loc="outside right upper", fontsize=11)
    legend.get_frame().set_facecolor("white")  # Set white background
    legend.get_frame().set_edgecolor("grey") 

    if resample is None:
        if isinstance(time, pd.DatetimeIndex) and len(time) > 0:
            time_str = time[0].strftime("%Y-%m")
        else:
            time_str = "default"
        resample = f"native-{time_str}"
    fn = f"ts-balance-{ylabel.replace(' ', '_')}-{resample}.pdf"
    plt.savefig(os.path.join(directory, fn))
    plt.close()

def process_carrier(group_item, balance, months, colors, config, output_dir):
    group, carriers = group_item
    if not isinstance(carriers, list):
        carriers = [carriers]

    mask = balance.index.get_level_values("bus_carrier").isin(carriers)
    df = balance[mask].groupby("carrier").sum().div(1e3).T

    kwargs = dict(
        ylabel=group,
        colors=colors,
        max_threshold=config["max_threshold"],
        mean_threshold=config["mean_threshold"],
        directory=output_dir,
    )

    if config["annual"]:
        plot_energy_balance_timeseries(df, resample=config["annual_resolution"], **kwargs)

# Load energy balance 
balance = n.statistics.energy_balance(aggregate_time=False, nice_names=False)

# Colors 
n.carriers["color"] = n.carriers["color"].replace("", "grey")
colors = n.carriers.color.copy()
colors["urban central HT-ATES charger"] = "#531ae4"
colors["urban central HT-ATES discharger"] = "#ff0004"

# Groups 
groups = config["carrier_groups"].copy()
groups.update({c: [c] for c in config["carriers"]})
available_carriers = set(n.buses.carrier.unique())
groups = {k: v for k, v in groups.items() if not set(v).isdisjoint(available_carriers)}

# Time slices 
snapshots = n.snapshots
months = snapshots.to_series().groupby(snapshots.to_series().dt.to_period("M")).apply(lambda x: x.index)

# Run and save new files
for group_item in tqdm(groups.items(), desc="Plotting groups"):
    process_carrier(group_item, balance, months, colors, config, output_dir)


INFO:pypsa.io:Imported network base_s_16__3H_2050-htates.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
Plotting groups: 100%|██████████| 10/10 [00:05<00:00,  1.85it/s]
